Exercises 1-2 of Climate Modelling Slides

Exercise 1 is an example
Exercise 2 is an independent work

Exercise 1: Linear Energy Balance Model
C(dT/dt) = S0/4(1-ap)-A-BT
where C(dT/dt) is the time evolution of the planet, S0/4 is the solar incoming short-wave radiation, a is the albedo, and A and B are the outgoing long-wave radiation

C=8.36x10^8 JK^-1m^-2, S0=1360 Wm^-2, αp
=0.30, A=-367.58 Wm^-2, B=2.09 Wm^-2K^-1

1) what is the equilibrium temperature for the given values of the parameters? What is the relaxation
time scale (in years)?
2) write a simple code to integrate numerically the equation. Plot time series of the solution starting
from different initial conditions;
3) repeat 2) for a few different cases changing the equilibrium temperature and/or the relaxation time
scale, by tuning the appropriate parameters.
4) compute and plot the potential of the model for a few different cases

1) what is the equilibrium temperature for the given values of the parameters? 

    a) Define the equilibrium temperature:
        At equilibrium, the change is 0 so dT/dt = 0
        Set everything on the right-hand side to 0, then solve for T to find the equilibrium temperature
        
        0 = S0/4(1-ap)-A-BT
        BT = S0/4(1-ap)-A
        T = (S0/4(1-ap)-A)/B

    b) Use given parameter values to solve for the equilibrium temperature (code below)

In [ ]:
#heat capacity
C=8.36*10**8
#incoming short-wave radiation
S_zero=1360
#albedo
Ap=0.30
#outgoing long-wave radiation
A=-367.58
B=2.09

#equilibrium temperature equation
eq_temp = ((S_zero/4)*(1-Ap)-A)/B
print(eq_temp)

1) What is the relaxation time scale (in years)?

    a) Define the deviation from the equilibrium:
    
        delta(T) = T-T0
        T=T0 + delta(T)

    c) Plug the difference into the original Energy Balance Model:
    
        C(d(T0+delta(T))/dt) = S0/4(1-ap)-A-B(T0+delta(T))
        The derivative of a constant is 0, so d(T0) cancels
        C(d(T0+delta(T))/dt) = S0/4(1-ap)-A-B(T0+delta(T))

        According to equilibrium temperature S0/4(1-ap)-A-BT = 0, so that also cancels
        C(d(delta(T))/dt) = -B(delta(T))

        Divide by C to get standard form:
        d(delta(T))/dt = -B(delta(T))/C
   
        Change notation to reflect standards in physics:
        B/C = 1/tau0
        tau0 = C/B

In [13]:
#relaxation timescale equation
tau_zero_seconds = C/B

#convert seconds to years
tau_zero = tau_zero_seconds/86400/365
print(tau_zero)

12.683916793505835


2) write a simple code to integrate numerically the equation. Plot time series of the solution starting
from different initial conditions

    a) Choose discretisation points for analysis (although time is continuous, choose discrete points on the number line to
    form plot of trajectory in time)